In [1]:
import yt_dlp
import os
import numpy as np
import librosa
import pandas as pd

In [2]:
# --- Paso 0: Configuración y Funciones Auxiliares ---
import gc

def extract_audio_features(file_path, song_id_txt, sr=None, hop_length=512):
    """
    Extrae características de un archivo de audio, sincronizadas con el beat.

    Args:
        file_path: Ruta al archivo de audio.
        song_id_txt: El ID de la canción (nombre del archivo .txt).
        sr: Frecuencia de muestreo (opcional).  Si es None, usa la original.
        hop_length:  Tamaño del salto (en samples) para características como MFCC.

    Returns:
        Diccionario con características sincronizadas con el beat y songId,
        o None si hay error.
    """
    try:
        y, sr = librosa.load(file_path, sr=sr, duration=120.0)

        # Separar componentes armónicos y percusivos
        y_harmonic, y_percussive = librosa.effects.hpss(y)

        # Seguimiento del beat (en la señal percusiva)
        tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

        # --- MFCCs ---
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
        mfcc_delta = librosa.feature.delta(mfcc)
        # Sincronizar con el beat (media)
        beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

        # --- Cromagrama ---
        chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=hop_length)
        # Sincronizar con el beat (mediana)
        beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)

        # --- Combinar Características ---
        beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

        # Convertir a lista y crear diccionario
        features = {
            'song_id': song_id_txt.replace(".txt", ""),
            'beat_features': beat_features.T.tolist()  # Transponer para tener (beats, features)
            # Puedes añadir más características aquí si quieres,
            #  pero DEBEN estar sincronizadas con el beat.
        }
        return features

    finally:
        # Limpieza
        del y, sr, y_harmonic, y_percussive, mfcc, mfcc_delta
        del beat_mfcc_delta, chromagram, beat_chroma, beat_features
        gc.collect()

In [ ]:
import os
import yt_dlp

folder = "Canciones2"
if not os.path.exists(folder):
    os.makedirs(folder)

def download_audio(search_query, output_filename):
    ydl_opts = {
        'cookiefile': 'cookies.txt',  # Archivo de cookies actualizado
        'default_search': 'ytsearch',
        'format': 'bestaudio/best',
        'outtmpl': os.path.join("Canciones2", output_filename + '.wav'),
        'extractor_args': {
            'youtube': {
                'player_client': ['web'],
            }
        },
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }],
        'ignore_no_formats_error': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([search_query])
        print(f"Descargado: {os.path.join('Canciones2', output_filename + '.wav')}")
    except Exception as e:
        print(f"Error al descargar {search_query}: {e}")

In [4]:
df = pd.read_csv("Data_completa_enrriquecida.csv")

In [5]:
df.head(5)

,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,dominance_tags,mbid,spotify_id,genre,cuadrante,cuadReal,lexicones,emociones,emocionesLetra,emocion_mas_comun
0,Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,...,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3,2,[0. 2. 0. 0. 0. 2. 6. 0. 0. 2.],"['anticipation', 'negative', 'positive', 'trust']","['3', '2', '1', '3']",3
1,Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,...,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3,2,[0. 6. 0. 0. 0. 6. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
2,Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,...,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3,2,[0. 2. 0. 0. 0. 2. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
3,Data/Letras/4-Bamboo-Banga-M.I.A..txt,en,complete,1.0,"Bamboo Banga Lyrics[Intro]\nRoad runner, road ...",road runner road runner going miles hour road ...,bamboo-banga-m.i.a..txt,spotify:track:6tqFC1DIOphJkCwrjVzPmg,Bamboo Banga,spotify:artist:0QJIPDAEDILuo8AIq3pMuU,...,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,3,1,[0. 0. 0. 0. 0. 0. 4. 0. 0. 0.],['positive'],['1'],1
4,Data/Letras/5-Die-MF-Die-Dope.txt,en,complete,1.0,Die MF Die Lyrics[Intro]\nDie!\n\n[Verse 1]\nI...,die need forgiveness need hate need acceptance...,die-mf-die-dope.txt,spotify:track:5bU4KX47KqtDKKaLM4QCzh,Die MF Die,spotify:artist:7fWgqc4HJi3pcHhK8hKg2p,...,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,3,1,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3


In [6]:
df.shape

(24966, 50)

In [7]:
def single_or_mode(series):
    """
    Si la columna tiene exactamente un único valor en el grupo, lo devuelve.
    De lo contrario, devuelve la moda (el valor más frecuente).
    En caso de no existir valor definido, devuelve None.
    """
    # Eliminamos NaN para evitar que interfieran
    sin_nan = series.dropna()
    
    # Valores únicos
    unique_vals = sin_nan.unique()
    
    if len(unique_vals) == 1:
        # Si hay un único valor, lo devolvemos
        return unique_vals[0]
    else:
        # Si hay más de un valor, calculamos la moda
        modos = sin_nan.mode()
        if len(modos) > 0:
            return modos.iloc[0]
        else:
            # Si no hay modo (puede estar vacío), retornamos None
            return None


In [8]:
# Función para manejar duplicados de songId
def consolidate_group(group):
    return pd.Series({
        'Idioma': single_or_mode(group['Idioma']),
        'lyrics_state': single_or_mode(group['lyrics_state']),
        'track_name':single_or_mode(group['track_name']),
        'artist_uri':single_or_mode(group['artist_uri']),
        'hasLetra': group['hasLetra'].max(),
        'Lyrics': max(group['Lyrics'], key=len),
        'Lyrics_proces': max(group['Lyrics_proces'], key=len),
        'track_uri': single_or_mode(group['track_uri']),
        'valence': single_or_mode(group['valence']),
        'valence_tags': single_or_mode(group['valence_tags']),
        'arousal_tags': single_or_mode(group['arousal_tags']),
        'emocion_mas_comun': single_or_mode(group['emocion_mas_comun']),
        'cuadrante': single_or_mode(group['cuadrante']),
        'cuadReal': single_or_mode(group['cuadReal'])
    })



df = df.groupby('songId').apply(consolidate_group).reset_index()
print("Número de songIds únicos después de consolidar:", df['songId'].nunique())

Número de songIds únicos después de consolidar: 24026


In [9]:
song_id_counts = df['songId'].value_counts()

duplicated_song_ids = song_id_counts[song_id_counts > 1].index

df[df['songId'].isin(duplicated_song_ids)].head(20)

,songId,Idioma,lyrics_state,track_name,artist_uri,hasLetra,Lyrics,Lyrics_proces,track_uri,valence,valence_tags,arousal_tags,emocion_mas_comun,cuadrante,cuadReal


In [10]:
df.head(5)

,songId,Idioma,lyrics_state,track_name,artist_uri,hasLetra,Lyrics,Lyrics_proces,track_uri,valence,valence_tags,arousal_tags,emocion_mas_comun,cuadrante,cuadReal
0,"""boy-next-door""-its-my-party.txt",en,complete,"""The Boy Next Door""",spotify:artist:3mYd6hxK1vk6puzhSp0X3D,1.0,8 ContributorsMake Noize Lyrics[Intro: Kid]\nT...,duos effect house party set catching wreck mic...,spotify:track:17CW8CsTByrCWb58btMkSX,0.9650,0.674101,0.121831,1,1,1
1,"""light-mass-prayers""-porcupine-tree.txt",en,complete,"""Light Mass Prayers""",spotify:artist:5NXHXK6hOCotCF8lvGM1I0,1.0,1 ContributorThe Laws Of Manu LyricsCHAPTER I....,chapter great sages approached manu seated col...,spotify:track:7IlzPutBp5cHQ2KTYWDnac,0.0258,0.079454,-0.284817,2,0,3
2,"""you-got-a-killer-scene-there,-man""-queens-of-...",en,complete,"""You Got A Killer Scene There, Man...""",spotify:artist:4pejUc4iciQfgdX6OKulQn,1.0,"10 Contributors“You Got A Killer Scene There, ...",mean obscene mob know hell mean knot tight bli...,spotify:track:6ZZiYOTFuZC1XLJjMiEnvS,0.4360,-0.173888,-0.402321,3,0,2
3,#1-animal-collective.txt,en,complete,#1,spotify:artist:4kwxTgCKMipBKhSnEstNKj,1.0,"#1 LyricsNoah: Mine, I want\nI caught you when...",noah want caught line got disconnected trouble...,spotify:track:6YZb2ESI1nvM2puafFrT7q,0.6780,0.653364,-0.007351,1,0,1
4,#1-must-have-sleater-kinney.txt,en,complete,#1 Must Have,spotify:artist:4wLIbcoqmqI4WZHDiBxeCB,1.0,#1 Must Have Lyrics[Verse 1]\nBearer of the fl...,bearer flag beginning believed riot girls cyni...,spotify:track:1GZou2dcibn0E0Y6mbONsj,0.9370,-0.114516,1.025842,3,0,1


In [11]:
import os
import numpy as np
import pandas as pd

# 1. Obtenemos la lista de archivos en la carpeta Canciones2
#    y la convertimos en un conjunto (set) para búsquedas rápidas.
files_in_folder = set(os.listdir("Canciones2"))

# 2. Creamos una nueva columna con el nombre de archivo esperado en Canciones2
df['audio_file'] = df['songId'].str.replace('.txt', '', regex=False) + '.wav.wav'

# 3. Filtramos el DataFrame para quedarnos solo con las canciones
#    cuyo archivo existe en la carpeta Canciones2.
df_filtered = df[df['audio_file'].isin(files_in_folder)].copy()

print(f"Número de canciones originales: {len(df)}")
print(f"Número de canciones con archivo en Canciones2: {len(df_filtered)}")

Número de canciones originales: 24026
Número de canciones con archivo en Canciones2: 1386


In [12]:
import os
import pandas as pd

# 1. Lista de archivos en la carpeta Canciones2
#    y la convertimos en un set para búsquedas rápidas.
existing_files = set(os.listdir("Canciones2"))

# 2. Crear una columna 'audio_file' con el nombre esperado de archivo
#    (por ejemplo, base_filename.wav)
df['audio_file'] = df['songId'].str.replace(".txt", "", regex=False) + ".wav.wav"

# 3. Filtrar para quedarnos solo con las que *no* estén en Canciones2
df_not_in_folder = df[~df['audio_file'].isin(existing_files)].copy()

print("Total de canciones que NO están en Canciones2:", len(df_not_in_folder))

# 4. De ese subset, queremos 400 canciones de cuadranteReal = 4
df_q4 = df_not_in_folder[df_not_in_folder['cuadReal'] == 4].sample(n=400, random_state=42)

# 5. 200 canciones de cuadranteReal = 3
df_q3 = df_not_in_folder[df_not_in_folder['cuadReal'] == 3].sample(n=200, random_state=42)

# 6. Unir ambos subsets
df_subset = pd.concat([df_q4, df_q3], ignore_index=True)
print("Canciones seleccionadas para descargar:", len(df_subset))


Total de canciones que NO están en Canciones2: 22640
Canciones seleccionadas para descargar: 600


In [13]:
df_subset.head(5)

,songId,Idioma,lyrics_state,track_name,artist_uri,hasLetra,Lyrics,Lyrics_proces,track_uri,valence,valence_tags,arousal_tags,emocion_mas_comun,cuadrante,cuadReal,audio_file
0,behind-your-eyes-jon-foreman.txt,en,complete,Behind Your Eyes,spotify:artist:5D3h9ZoobhetjXw3dKhcaq,1.0,3 ContributorsBehind Your Eyes Lyrics[Verse 1]...,let feelings dear scary find find street dear ...,spotify:track:5GOYJmKP4yY1fWzpkDT42j,0.824,0.496724,-1.001720,3,1,4,behind-your-eyes-jon-foreman.wav.wav
1,geriatric-punk-rock-boyfriend-kitten-on-keys.txt,en,complete,Geriatric Punk Rock Boyfriend,spotify:artist:4wNkV2ccopdr6WCuYyr3QW,1.0,True Romance - Screenplay LyricsTrue Romance\n...,true romance screenplay quentin tarantino tire...,spotify:track:7fYeet2X8k5o9UHkla8NQg,0.860,1.495306,0.927643,2,1,4,geriatric-punk-rock-boyfriend-kitten-on-keys.w...
2,wordless-chorus-my-morning-jacket.txt,en,complete,Wordless Chorus,spotify:artist:43O3c6wewpzPKwVaGEEtBM,1.0,Wordless Chorus Lyrics[Verse 1]\nSo much going...,going days forget instinct pays pleasure smile...,spotify:track:6xAa0kGkTLU22ZyoHOCHgi,0.779,0.759900,-0.304518,1,0,4,wordless-chorus-my-morning-jacket.wav.wav
3,havoc-in-heaven-jesca-hoop.txt,en,complete,Havoc in Heaven,spotify:artist:0fqE57gXXTTqxXlFYVNG2u,1.0,Havoc In Heaven Lyrics[Verse 1]\nRed ribbon on...,red ribbon ride carry horizon sun setting nigh...,spotify:track:1c76bCoHAenJv2ZmviO6r9,0.658,-0.727161,1.508174,1,0,4,havoc-in-heaven-jesca-hoop.wav.wav
4,strawberries-smooth.txt,en,complete,Strawberries,spotify:artist:7H6YbDjzY80v100UkfvAkx,1.0,1 ContributorStrawberries Lyrics[Intro]\nComin...,coming yeah huh strawberries hennessy yeah fem...,spotify:track:7rtGt7IE46Y9bxymlGnNqX,0.609,1.048858,1.538500,1,0,4,strawberries-smooth.wav.wav


In [ ]:
# Iterar sobre cada registro para descargar el audio.
# Se asume que el campo 'songId' en el DataFrame tiene un valor similar a "till-i-collapse-eminem.txt"
# Para el nombre del archivo de audio, se remueve la extensión ".txt" y se añade ".wav"

for index, row in df_subset.iterrows():
    # Construir la consulta de búsqueda (puedes ajustar la query para mejorar la precisión)
    query = f"{row['track_name']} {row['artist_uri']}"
    # Remover la extensión ".txt" para el nombre del archivo de audio
    base_filename = row['songId'].replace(".txt", "")
    print(f"Descargando '{query}' como {base_filename}.wav")
    download_audio(query, base_filename)


In [ ]:
del df

In [ ]:
import os
import csv

with open("features_extracted.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # Cabecera (columnas) - ajusta a tus necesidades
    writer.writerow(["song_id", "beat_features"])

    # Recorre solo las canciones que existen en Canciones2
    for index, row in df_subset.iterrows():
        audio_path = os.path.join("Canciones2", row['audio_file'])
        print(f"Extrayendo características de {audio_path}")
        features = extract_audio_features(audio_path, row['songId'])

        if features:
            # Escribe una fila con el ID y la lista/matriz de beat_features
            # Si beat_features es un array grande, podrías convertirlo a string/JSON
            writer.writerow([features["song_id"], features["beat_features"]])

        # Libera memoria de la variable 'features'
        del features

In [ ]:
# Convertir el diccionario a DataFrame, donde cada fila corresponde a una canción y sus beat_features
# Dado que las beat_features son listas (por beat), se puede guardar como objeto o procesar adicionalmente
features_df = pd.DataFrame.from_dict(features_dict, orient='index')
features_df.index.name = 'song_id'
features_df.rename(columns={0: "beat_features"}, inplace=True)  # Si se desea renombrar la única columna

In [ ]:
#--- Paso 3: Integración con el Dataset Original ---

# Si en tu DataFrame original el campo 'songId' tiene la extensión ".txt", se elimina para hacer match
df_filtered['song_id'] = df_filtered['songId'].apply(lambda x: x.replace(".txt", ""))

# Unir el DataFrame de características con el dataset original usando 'song_id'
df_final = df_filtered.merge(features_df, on='song_id', how='left')

# Mostrar las primeras filas del DataFrame final para verificar la unión
df_final.head(10)

In [ ]:
df_final.to_csv("Data_lyrics_audio.csv", index=False)